In [55]:
# !pip install PublicDataReader --upgrade

In [56]:
# import imp
# import PublicDataReader
# from PublicDataReader import TransactionPrice
# imp.reload(PublicDataReader)

# service_key = "eg0rzIVzjLzLU0k%2BmCHf4gTyKxC4%2BhhxB6LzV%2BG7bjEEo%2BnlqCRFoA%2FefD46KKppluWdHWCflLac1wFd0cdTUg%3D%3D"
# api = PublicDataReader.TransactionPrice(service_key)

# # # 단일 월 조회
# # df = api.get_data(
# #     property_type="아파트",
# #     trade_type="매매",
# #     sigungu_code="11650",
# #     year_month="202212",
# # )

# # 기간 내 조회
# df = api.get_data(
#     property_type="아파트",
#     trade_type="매매",
#     sigungu_code="28185",
#     start_year_month="200001",
#     end_year_month="202501",
# )

# df.tail()


In [50]:
import sqlite3
import requests
from datetime import datetime, date, timedelta
import hashlib
import xml.etree.ElementTree as ET
import time

# SQLite 데이터베이스 생성 및 연결
conn = sqlite3.connect('RealEstateTransactions.db')
cursor = conn.cursor()

# 테이블 생성
cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        hash TEXT UNIQUE,
        transaction_amount TEXT,
        construction_year TEXT,
        year TEXT,
        month TEXT,
        day TEXT,
        legal_dong TEXT,
        apartment_name TEXT,
        exclusive_area TEXT,
        jibun TEXT,
        regional_code TEXT,
        floor TEXT
    )
''')
conn.commit()

# 해시 생성 함수
def generate_hash(data):
    hash_input = "".join([data[k] for k in sorted(data.keys())])
    return hashlib.sha256(hash_input.encode()).hexdigest()

# API 호출 및 데이터 저장 함수 정의
def fetch_and_store_data(year, month, lawd_cd='28185'):
    base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"
    service_key = "{SECRET_KEY}"  # 실제 서비스 키 사용 필요
    deal_ymd = f"{year}{month:02d}"
    url = f"{base_url}?serviceKey={service_key}&LAWD_CD={lawd_cd}&DEAL_YMD={deal_ymd}"
    response = requests.get(url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        for item in root.iter('item'):
            data = {
                'transaction_amount': item.findtext('거래금액').strip(),
                'construction_year': item.findtext('건축년도'),
                'year': item.findtext('년'),
                'month': item.findtext('월'),
                'day': item.findtext('일'),
                'legal_dong': item.findtext('법정동').strip(),
                'apartment_name': item.findtext('아파트'),
                'exclusive_area': item.findtext('전용면적'),
                'jibun': item.findtext('지번'),
                'regional_code': item.findtext('지역코드'),
                'floor': item.findtext('층')
            }

            hash_value = generate_hash(data)
            # 중복 검사
            cursor.execute('SELECT hash FROM transactions WHERE hash = ?', (hash_value,))
            if cursor.fetchone() is None:
                # 중복 없으면 삽입
                data['hash'] = hash_value
                cursor.execute('''
                    INSERT INTO transactions (
                        hash, transaction_amount, construction_year, year, month, day, 
                        legal_dong, apartment_name, exclusive_area, jibun, regional_code, floor
                    ) VALUES (
                        :hash, :transaction_amount, :construction_year, :year, :month, :day, 
                        :legal_dong, :apartment_name, :exclusive_area, :jibun, :regional_code, :floor
                    )
                ''', data)
                conn.commit()

# 날짜 범위 설정 및 데이터 수집
start_date = date(2000, 1, 1)
end_date = date(2024, 4, 1)

current_date = start_date
while current_date <= end_date:
    print(current_date)
    year = current_date.year
    month = current_date.month
    fetch_and_store_data(year, month)  # 데이터 수집 및 저장
    current_date += timedelta(days=32)  # 다음 달로 이동
    current_date = current_date.replace(day=1)  # 달의 첫 날로 조정
    time.sleep(1)

# 데이터베이스 연결 종료
conn.close()


2000-01-01
2000-02-01
2000-03-01
2000-04-01
2000-05-01
2000-06-01
2000-07-01
2000-08-01
2000-09-01
2000-10-01
2000-11-01
2000-12-01
2001-01-01
2001-02-01
2001-03-01
2001-04-01
2001-05-01
2001-06-01
2001-07-01
2001-08-01
2001-09-01
2001-10-01
2001-11-01
2001-12-01
2002-01-01
2002-02-01
2002-03-01
2002-04-01
2002-05-01
2002-06-01
2002-07-01
2002-08-01
2002-09-01
2002-10-01
2002-11-01
2002-12-01
2003-01-01
2003-02-01
2003-03-01
2003-04-01
2003-05-01
2003-06-01
2003-07-01
2003-08-01
2003-09-01
2003-10-01
2003-11-01
2003-12-01
2004-01-01
2004-02-01
2004-03-01
2004-04-01
2004-05-01
2004-06-01
2004-07-01
2004-08-01
2004-09-01
2004-10-01
2004-11-01
2004-12-01
2005-01-01
2005-02-01
2005-03-01
2005-04-01
2005-05-01
2005-06-01
2005-07-01
2005-08-01
2005-09-01
2005-10-01
2005-11-01
2005-12-01
2006-01-01
2006-02-01
2006-03-01
2006-04-01
2006-05-01
2006-06-01
2006-07-01
2006-08-01
2006-09-01
2006-10-01
2006-11-01
2006-12-01
2007-01-01
2007-02-01
2007-03-01
2007-04-01
2007-05-01
2007-06-01
2007-07-01

In [53]:
conn = sqlite3.connect('RealEstateTransactions.db')
cursor = conn.cursor()
# SELECT * FROM transactions limit 10
result = cursor.execute('SELECT * FROM transactions order by year asc, month asc limit 10').fetchall()

In [54]:
result

[(1,
  'f56b0312f73c4a209c0e2e54311f82b7eae6614d82f46e3ed74885096129ca2e',
  '19,250',
  '1996',
  '2006',
  '1',
  '5',
  '옥련동',
  '현대3',
  '84.6',
  '635-1',
  '28185',
  '12'),
 (2,
  '91ff3a7c05aeb6502039351b6551cb530d915709fd5b7c6d81715307083796ae',
  '11,550',
  '1997',
  '2006',
  '1',
  '6',
  '옥련동',
  '현대4',
  '59.985',
  '637-1',
  '28185',
  '3'),
 (3,
  '06b81cb0ff5b6dfc8283f421a8314696e9b8121cdeec592ebde7aab9bc680288',
  '10,900',
  '1997',
  '2006',
  '1',
  '10',
  '옥련동',
  '현대5',
  '59.985',
  '639-1',
  '28185',
  '2'),
 (4,
  '380bf9056ab9a8a0436cada9b0b45cdabc32435fb0ea21a4e37c8ab14cd3b462',
  '17,000',
  '1997',
  '2006',
  '1',
  '14',
  '옥련동',
  '현대5',
  '84.6',
  '639-1',
  '28185',
  '8'),
 (5,
  'ac649a8a6d6ce98021e54bb4c1b60935697cb109b4d87daf5246d7421773bee5',
  '4,980',
  '1997',
  '2006',
  '1',
  '15',
  '옥련동',
  '아주1',
  '29.7',
  '640-1',
  '28185',
  '13'),
 (6,
  'ad77a190b3e065cffa3839155d49b250f3cf8d8d0d45a41fb84510650894166a',
  '11,800',
  '1997',
